In [15]:
# Nurse scheduling problem
# Authors: Team JAMS: Andrea Henshall, Azza Fadhel, Maninder Kaur, Srikar Vanavasma
# Purpose: Schedules given number of nurses over a given number of schedule days and shifts
#          Ensures nurses are not scheduled for more than 1 shift per day
#          Minimizes the shift changes for a nurse from 1 day to the next
# Created on: 31 January, 2021
# Referenced:         https://github.com/dwave-examples/nurse-scheduling
# And:                Gottlieb, Joel, "Practical Quantum Computing: Programming"
#                     MIT Hackathon/iQUise iQuHACK 2021, January 29, 2021

from dimod import DiscreteQuadraticModel
from dwave.system import LeapHybridDQMSampler, LeapHybridSampler
import networkx as nx
from copy import deepcopy

token = # insert token here

scheduleDays = 7
numShifts = 3
numNurses = 4

multipleShiftsCost = 10
shiftChangeCost = 1

### Helper programs
# returns composite index given (shift, day)
def Get_Index(shift, day):
    return shift * scheduleDays + day

# Inverse of get_index - given a composite index in a 1D list, return the
# nurse_index and day_index
def Get_Shift_and_Day(index):
    shift, day = divmod(index, scheduleDays)
    return shift, day

graphSize = numShifts*scheduleDays
nurses = range(numNurses)
shiftList = list(range(numShifts))

dqm = DiscreteQuadraticModel()

# Add nodes with number of options each can have
for node in range(graphSize):
    dqm.add_variable(numNurses, label = node)

# Add quadratic costs
for node in range(graphSize):
    isDayAfter = False
    otherShifts = deepcopy(shiftList)
    
    shift, day = Get_Shift_and_Day(node)
    otherShifts.remove(shift)
    dayAfter = day + 1

    if day + 1 < scheduleDays:
        isDayAfter = True
        
    # Don't schedule the same person for multiple shifts
    for ii in otherShifts:
        index = Get_Index(ii, day)
        dqm.set_quadratic(node, index, {(n, n): multipleShiftsCost for n in nurses})
            
    # Don't change a nurse's schedule from 1 day to another
    if isDayAfter:
        for ii in otherShifts:
            index = Get_Index(ii, dayAfter)
            dqm.set_quadratic(node, index, {(n, n): shiftChangeCost for n in nurses})
            
# Solve the problem            
sampler = LeapHybridDQMSampler(endpoint='https://cloud.dwavesys.com/sapi/', token=token, solver={ 'name': 'hybrid_discrete_quadratic_model_version1'})
sampleset = sampler.sample_dqm(dqm)

sample = sampleset.first.sample
energy = sampleset.first.energy
valid = True

# Check no nurses are scheduled for more than 1 shift per day
for thisDay in range(scheduleDays):
    for thisShift in range(numShifts - 1):
        index1 = Get_Index(thisShift, thisDay)
        index2 = Get_Index(thisShift + 1, thisDay)
        if sample[index1] == sample[index2]:
            valid = False
            break

# Print results
print("Solution energy: ", energy)

for ii in range(numShifts):
    str_row = ""
    for jj in range(scheduleDays):
        thisIndex = Get_Index(ii, jj)
        str_row += str(sample[thisIndex]) + ' '
    print(str_row)

print("Solution validity: ", valid)


Solution energy:  0.0
1 1 1 1 3 3 3 
3 3 3 2 2 1 2 
0 0 0 0 0 0 0 
Solution validity:  True
